<a href="https://colab.research.google.com/github/hanbun-pom/siosio/blob/master/Yukapedia_%E3%83%91%E3%83%AF%E3%83%97%E3%83%AD%E7%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#君たちが打ったコメントを検索できます

不特定多数への公開は今の所ご遠慮下さい  
当ページ外で行われた操作についての責任は負いかねます  
@hanbun_pom
    
####🌸使い方🌸  
① 検索条件を入力  
② ↓にカーソルを合わせると表示される ▶ を押して実行

In [3]:
import pandas as pd
import datetime
import requests

#@markdown ---
名前 = ""  #@param {type:"string"}
名前完全一致 = False  #@param {type:"boolean"} 
名前正規表現 = False  #@param {type:"boolean"} 
#@markdown ---
コメント = "\u6751\u7530"  #@param {type:"string"}
コメント完全一致 = False  #@param {type:"boolean"}
コメント正規表現 = True  #@param {type:"boolean"} 
#@markdown ---
t秒前から再生 = 30 #@param {type:"slider", min:0, max:60, step:5}
#@markdown ---
配信日_以降 = '2019-12-24' #@param {type:"date"}
配信日_以前 = '2040-04-01' #@param {type:"date"}　


try:
    df2

except NameError:

    # 配信一覧　取得
    youtube = 'https://youtu.be/'
    master = 'https://raw.githubusercontent.com/hanbun-pom/siosio/master/'
    videos = pd.read_csv(master + 'videos_pawapuro.csv')

    # コメント一覧　作成
    df_list=[]
    for Id in videos['videoId']:
        csvURL = master + Id + '.tsv'
        res = requests.get(csvURL)
        if res.status_code == 200:
            tmp_df = pd.read_table(csvURL, header=None, names=['time','name','comment','videoId'],quotechar='"', quoting=3)
            tmp_df['videoId']= Id
            df_list.append(tmp_df)
        else:
            pass
    df = pd.concat(df_list, ignore_index=True)

    # 配信タイトル、公開時間
    df = df.merge(videos)
    df = df.rename(columns={'publishedAt': 'DateTime'})
    df['DateTime'] = pd.to_datetime(df['DateTime']).dt.date

    # 再生時間指定URL
    trans = str.maketrans({':': '',',': ''})
    s = df['time'].str.translate(trans).astype(int)
    s = s-40*(abs(s)//100)-2400*(abs(s)//10000)-t秒前から再生
    s = ((abs(s)+s)//2).astype(str)

    df['videoId'] = youtube + df['videoId'] + '?t=' + s + 's'

else:
    pass

#名前、コメント検索
def query_condition(data, col, query, is_perfect, is_regex):
    if is_perfect:
        return data[col].str.match(query) if is_regex else data[col] == query
    else:
        return data[col].str.contains(query, na=False, regex=is_regex)

name_condition = query_condition(df, "name", 名前, 名前完全一致, 名前正規表現)
comment_condition = query_condition(df, "comment", コメント, コメント完全一致, コメント正規表現)

df2 = df[name_condition & comment_condition].copy()

# 配信日指定
start = datetime.datetime.strptime(配信日_以降, '%Y-%m-%d').date()
end = datetime.datetime.strptime(配信日_以前, '%Y-%m-%d').date()
df3 = df2[df2['DateTime'] >= start]
df3 = df3[df3['DateTime'] <= end]

pd.set_option('display.max_rows', 10000)
df3


,time,name,comment,videoId,title,DateTime
2,-0:58,雨宮恵,そういう事だね村田すごい,https://youtu.be/I5MSGbOF9aM?t=0s,【パワプロ2020 #6】楠坂高校の新しい夏が始まる！栄冠ナイン⚾✨【#楠栞桜】,2020-08-09
7,-0:56,テーク,卍村田卍,https://youtu.be/I5MSGbOF9aM?t=0s,【パワプロ2020 #6】楠坂高校の新しい夏が始まる！栄冠ナイン⚾✨【#楠栞桜】,2020-08-09
8,-0:54,デへへ,村田最強！,https://youtu.be/I5MSGbOF9aM?t=0s,【パワプロ2020 #6】楠坂高校の新しい夏が始まる！栄冠ナイン⚾✨【#楠栞桜】,2020-08-09
9,-0:54,氷水,村田はもういない、いないんだ！,https://youtu.be/I5MSGbOF9aM?t=0s,【パワプロ2020 #6】楠坂高校の新しい夏が始まる！栄冠ナイン⚾✨【#楠栞桜】,2020-08-09
12,-0:53,あきら,村田は強いからね,https://youtu.be/I5MSGbOF9aM?t=0s,【パワプロ2020 #6】楠坂高校の新しい夏が始まる！栄冠ナイン⚾✨【#楠栞桜】,2020-08-09
13,-0:53,芋下きぬ,村田は化け物投手,https://youtu.be/I5MSGbOF9aM?t=0s,【パワプロ2020 #6】楠坂高校の新しい夏が始まる！栄冠ナイン⚾✨【#楠栞桜】,2020-08-09
15,-0:50,山田太一,村田マジエース 魔物強い,https://youtu.be/I5MSGbOF9aM?t=0s,【パワプロ2020 #6】楠坂高校の新しい夏が始まる！栄冠ナイン⚾✨【#楠栞桜】,2020-08-09
19,-0:48,ポチムゴン,転生村田はピッチャーの中でも最上級だから,https://youtu.be/I5MSGbOF9aM?t=0s,【パワプロ2020 #6】楠坂高校の新しい夏が始まる！栄冠ナイン⚾✨【#楠栞桜】,2020-08-09
20,-0:47,xxzxcuzx,村田最強！村田最強！,https://youtu.be/I5MSGbOF9aM?t=0s,【パワプロ2020 #6】楠坂高校の新しい夏が始まる！栄冠ナイン⚾✨【#楠栞桜】,2020-08-09
21,-0:46,si n,村田最強！,https://youtu.be/I5MSGbOF9aM?t=0s,【パワプロ2020 #6】楠坂高校の新しい夏が始まる！栄冠ナイン⚾✨【#楠栞桜】,2020-08-09


2020/03/19 追加　完全一致検索  正規表現検索　動画URL生成  
2020/04/07 変更　GitHub版  